In [1]:
import pandas as pd
from datetime import datetime
import calendar

In [2]:
df = pd.read_csv("sample_data.csv")
df

,MemberID,Measure,ActualDate,Value
0,1001,HbA1c,15-06-2025,7.2
1,1001,HbA1c,06-2025,6.9
2,1002,eGFR,2024,85
3,1003,BP,10-03-2025,120/80
4,1004,PHQ-9,03-2025,12
5,1005,Colorectal Cancer Screening,2016,Done
6,1006,Breast Cancer Screening,02-2024,Done
7,1007,Osteoporosis Screening,2025,Done
8,1008,HEENT Exam,14-07-2025,Completed
9,1009,HbA1c,15-13-2025,8.0


In [3]:
MEASUREMENT_YEAR = 2025

def derive_modified_date(actual_date):
    try:
        if pd.isna(actual_date) or actual_date == "":
            return None, "Missing Actual Date"

        actual_date = str(actual_date)

        if len(actual_date) == 10:  
            return datetime.strptime(actual_date, "%d-%m-%Y"), None # day-month-year

        elif len(actual_date) == 7:  # month - year
            month, year = map(int, actual_date.split("-"))
            last_day = calendar.monthrange(year, month)[1]
            return datetime(year, month, last_day), None

        elif len(actual_date) == 4:  # year
            return datetime(int(actual_date), 12, 31), None

        else:
            return None, "Invalid date format"

    except:
        return None, "Invalid date value"


In [4]:
def validate_record(row):
    errors = []

    if pd.isna(row["MemberID"]):
        errors.append("Missing Member ID")

    modified_date, date_error = derive_modified_date(row["ActualDate"])
    if date_error:
        errors.append(date_error)

    if modified_date:
        if modified_date.year > MEASUREMENT_YEAR:
            errors.append("Date beyond measurement year")

    return modified_date, " | ".join(errors) if errors else "VALID"

In [5]:
results = []

for _, row in df.iterrows():
    mod_date, status = validate_record(row)
    results.append({
        "MemberID": row["MemberID"],
        "Measure": row["Measure"],
        "ActualDate": row["ActualDate"],
        "ModifiedDate": mod_date,
        "ValidationStatus": status
    })

output_df = pd.DataFrame(results)
output_df

,MemberID,Measure,ActualDate,ModifiedDate,ValidationStatus
0,1001,HbA1c,15-06-2025,2025-06-15,VALID
1,1001,HbA1c,06-2025,2025-06-30,VALID
2,1002,eGFR,2024,2024-12-31,VALID
3,1003,BP,10-03-2025,2025-03-10,VALID
4,1004,PHQ-9,03-2025,2025-03-31,VALID
5,1005,Colorectal Cancer Screening,2016,2016-12-31,VALID
6,1006,Breast Cancer Screening,02-2024,2024-02-29,VALID
7,1007,Osteoporosis Screening,2025,2025-12-31,VALID
8,1008,HEENT Exam,14-07-2025,2025-07-14,VALID
9,1009,HbA1c,15-13-2025,NaT,Invalid date value


In [6]:
output_df.to_csv("validation_output.csv", index=False)
print("Validation output saved successfully.")

Validation output saved successfully.
